<a href="https://colab.research.google.com/github/yhatpub/yhatpub/blob/notebook/notebooks/fastai/lesson2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fastai Lesson 2 on YHat.pub

This notebook picks up from [Fastai Fastbook 02_production](https://github.com/fastai/fastbook/blob/master/02_production.ipynb) publishing your bear classifier to [YHat.pub](https://yhat.pub)

In [ ]:
!pip install -q --upgrade --no-cache-dir torch torchvision torchaudio
!pip install -q --upgrade --no-cache-dir fastai

In [ ]:
from fastai.vision.all import *

In [ ]:
!pip install -Uqq --no-cache-dir git+https://github.com/yhatpub/yhat_params.git@main

In [ ]:
from yhat_params.yhat_tools import inference_test, FieldType, inference_predict

In [ ]:
!wget -q --no-check-certificate 'https://docs.google.com/uc?export=download&id=1tRoH4Yxg8izIlX7bCrRTwS13wHb0kiJT' -r -A 'uc*' -e robots=off -nd
!mkdir -p model
!mv $(ls -S uc* | head -1) ./model/export.pkl

In [ ]:
!ls -l ./model/export.pkl

In [ ]:
learn_inf = load_learner('./model/export.pkl')

In [ ]:
input = {"image": FieldType.PIL}
output = {"text": FieldType.Text}

@inference_predict(input=input, output=output)
def predict(params):
    img = PILImage.create(np.array(params["image"].convert("RGB")))
    result = learn_inf.predict(img)
    return {"text": str(result[0])}

In [ ]:
from yhat_params.yhat_tools import in_colab

if in_colab():
    import urllib.request
    from PIL import Image
    urllib.request.urlretrieve("https://upload.wikimedia.org/wikipedia/commons/thumb/a/a9/GrizzlyBearJeanBeaufort.jpg/220px-GrizzlyBearJeanBeaufort.jpg", "input_image.jpg")
    img = Image.open("input_image.jpg")
    inference_test(predict_func=predict, params={'image': img})